<a href="https://colab.research.google.com/github/mankicom/DEV_GDPS_TEMP_LSTM/blob/master/KMEANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 지점개수 축약을 위한 군집분류

### **모듈 로드**

*   데이터 핸들 기본모듈: numpy, pandas, math, time, os, sys, copy
*   데이터 정규화 모듈: MinMaxScaler
*   K평균 분류 모듈: KMeans
*   사용자 정의 모듈: tran_data_load_stnall_y(관측자료 로드 함수), find_stn_idx: 지점번호 idx 호출








In [10]:
#-------------------------------------------------------------------------
# .. Module load

#.. module
import numpy as np
import pandas as pd
import math
from time import time
import os
import sys
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import copy

#.. local
sys.path.insert(0, '/content/kmeans/inc')
from tran_data_load_stnall_y import tran_data_load_stnall_y
from test_find_stnidx import find_stn_idx
#import read_namelistf90


### **데이터 세트 설정**


*   입력 데이터의 경로, 파일명, 차원 정보 설정
*   kmeans 설정



In [12]:
#-------------------------------------------------------------------------
# .. Data set

data_dir = '/content/kmeans/DAIN/' # 입력자료 경로
tran_data_per = "2016050100-2021043000-24-1605-2104" # 입력자료 파일의 기간명
obs_stn_list = '/content/kmeans/DABA/new_dfs_merg_station_directory_2021050100.dat' # 지점정보 파일

num_fct = 136     # 입력자료 예측시각 개수
num_his = 1826    # 입력자료 표본개수
num_ele = 2       # 입력자료 변수 개수 [u,v] 2개
element = 'uvw'   # 입력자료 파일의 요소명

# kmeans 분류개수 설정 및 결과 파일이름 설정
n_clst = 4       
out_file = '/content/kmeans/DAOU/kmean_k'+str(n_clst)+'out_nor_'+ element + '_' + tran_data_per + '_00utc'

### **관측지점 정보 업로드**


*   동네예보용 지점정보 형식 자료 로드 및 위경도, 지점번호, 지점개수 정보 획득



In [ ]:
#-------------------------------------------------------------------------
# .. OBS Station load

print ("2. Read OBS station list")
exists = os.path.isfile(obs_stn_list)
if exists:
   obs_stn_id  = pd.read_fwf(obs_stn_list, delimiter=' ', header=2, usecols=[0])
   obs_stn_id  = np.array(obs_stn_id,  dtype=np.int)
   obs_stn_lat  = pd.read_fwf(obs_stn_list, delimiter=' ', header=2, usecols=[3])
   obs_stn_lat  = np.array(obs_stn_lat,  dtype=np.float)
   obs_stn_lon  = pd.read_fwf(obs_stn_list, delimiter=' ', header=2, usecols=[4])
   obs_stn_lon  = np.array(obs_stn_lon,  dtype=np.float)
   num_stn = len(obs_stn_id)
else:
   sys.exit("STOP Error: Could not found : "+ obs_stn_list)
#print(obs_stn_id)

# .. Find stn_id_idx
#run_stn_idx = find_stn_idx(obs_stn_id, run_stn_id)

### **입력자료 로드**


*   관측 U,V 자료 로드
*   -999. 값은 np.nan으로 대체
*   자료가공 편의를 위해 업로드한 입력자료의 차원순서 변경 및 확인






In [ ]:
#-------------------------------------------------------------------------
# .. Fcst load :  data dim( input_size, num_stn, num_his, num_fct )
#
#    Trainining data used for cross-validation
#    Test data used to evaluate best model out of randomized search
#

print ("3. Training data load")
tran_y = tran_data_load_stnall_y( data_dir, tran_data_per, element,
                                  num_ele, num_stn, num_his, num_fct )

np.place(tran_y, tran_y==-999., np.nan)
tran_y = np.swapaxes(tran_y, 0, 3)
input_size = tran_y.shape[3]

print("loaded data shape: ", tran_y.shape)

input_size = tran_y.shape[3]
print ('tran_y shape= ', tran_y.shape)    # sequence, stn, batch, feature

### **데이터 정규화 및 평균**

kmeans 입력자료 생성을 위한 데이터 전처리1: [지점별 기간평균, 변수] 차원으로 가공하기

*   drop nan for nomalize for minmax scaler input
> tran_y의 결측 제거 및 정규화를 위해 먼저 4차원[시간개수,지점개수,표본개수,변수개수] 변수를 2차원[*,변수개수]로 치환하여 새로운 변수 fit_tran_y로 복사.
> numpy object인 fit_tran_y를 pandas object로 바꿔 pandas.DataFrame의 dropna(표본개수,특징)를 사용해 결측 행 제거. 

*   get restorator with obs range
> 정규화 모듈 MinMaxScaler object 생성(obs_scaler) 및 fit_tran_y 피팅--> 피팅한 자료의 요소별 최대최소값 정보 산출

*   feature nomalize
> 지점을 분류하기 때문에 [각 지점의 평균값, 변수] 차원의 데이터를 만들어야 한다. 정규화 최종산출물 선언(avg_nor_tran_y[지점,변수]) 및 초기화 하고, for 문을 이용해 지점별로 결측을 제거하고, 정규화 한 뒤, [지점,변수] 차원을 제외한 나머지의 평균값을 구한다.








In [ ]:
#-------------------------------------------------------------------------
# .. Normalize & mean

# .. drop nan for normalize for minmax scaler input
tr_q, tr_s, tr_b, tr_f = tran_y.shape[0], tran_y.shape[1], tran_y.shape[2], tran_y.shape[3]
fit_tran_y = copy.deepcopy( tran_y.reshape(tr_q*tr_s*tr_b, tr_f) )
print('before dropna, fit_tran_y shape: ', fit_tran_y.shape)
fit_tran_y = pd.DataFrame(fit_tran_y)
fit_tran_y = fit_tran_y.dropna()
print('after dropna, fit_tran_y shape: ', fit_tran_y.shape)

# .. restorator fitting
obs_scaler = MinMaxScaler()   # copy default true
obs_scaler.fit(fit_tran_y)


# .. feature normalize   ( train seq, feature = test seq, feature )
avg_nor_tran_y = np.ndarray( shape=(num_stn,tr_f), dtype=np.float32 )
avg_nor_tran_y.fill(np.nan)
print("-------Normalizing each stn")
for i in range(num_stn):
      mis_idx = np.unique( np.argwhere(np.isnan(tran_y[:,i,:,:]))[:,1] )   # num_his-n
      print(mis_idx.shape[0], tr_b)
      if mis_idx.shape[0]==tr_b: continue
      rem_tran_y = np.delete( tran_y[:,i,:,:], mis_idx, axis=1 )   # f, num_his-n, num_fct
      rem_tr_q, rem_tr_b, rem_tr_f = rem_tran_y.shape[0], rem_tran_y.shape[1], rem_tran_y.shape[2]
      nor_rem_tran_y = obs_scaler.transform(rem_tran_y.reshape(rem_tr_q*rem_tr_b, rem_tr_f))
      print("nor_rem_tran_y shape: ", nor_rem_tran_y.shape)
      avg_nor_tran_y[i,:] = np.nanmean( nor_rem_tran_y, axis=0 )

### **지점 필터**

kmeans 입력자료 생성을 위한 데이터 전처리2: 표본개수가 1년이 안되는 지점은 분석에서 제외

*   분석지점 중 표본 개수가 365개 미만의 지점은 분석에서 제외
> for 문을 통해 지점별로 표본의 개수가 365개 미만인 지점의 idx를 'missing_idx' 변수에 모아 위에서 구했던 avg_nor_tran_y 변수에서 그 지점들을 제거한다-->re_avg_nor_tran_y. 이와 동시에 지점번호, 위경도 정보를 똑같은 순서로 매칭을 해준다. 





In [ ]:
#-------------------------------------------------------------------------
# .. drop nan

print ('---------- Final training data shape before remove missing')
print ('avg_nor tran obs : ', avg_nor_tran_y.shape)
#print ('avg_tran obs : ', avg_tran_y)


## .. filter: avg_nor = -999.
#missing_idx = np.where( np.isnan(avg_nor_tran_y) )
#print ( missing_idx )

# .. filter: total - missing day < 365
mis_obs_thr = []   # for station
mis_tred = 365
for i in range(num_stn):
    mis_obs_idx = np.unique( np.argwhere( np.isnan(tran_y[:,i,:,0]) )[:,1] )

    if num_his - len(mis_obs_idx) < mis_tred:
       print ( i, obs_stn_id[i], " obs: {}".format(mis_obs_idx.shape), num_his-len(mis_obs_idx) )
       mis_obs_thr.extend([i])
missing_idx = sorted(set(mis_obs_thr))
print("filtering station(total - missing day <365)")
print(missing_idx)
print("missing len: ", len(missing_idx))
print("total-missing: ", num_stn - len(missing_idx))


re_avg_nor_tran_y = np.delete( avg_nor_tran_y, missing_idx, axis=0 )
re_obs_stn_id = np.delete( obs_stn_id, missing_idx, axis=0 )
re_obs_stn_lat = np.delete( obs_stn_lat, missing_idx, axis=0 )
re_obs_stn_lon = np.delete( obs_stn_lon, missing_idx, axis=0 )

print ('---------- Final training data shape after remove missing')
print ('re_avg_tran obs : ', re_avg_nor_tran_y.shape)

### **k-means 분류 수행**


*   KMeans 함수 설정 및 분류 수행: 분류개수(n_clusters)를 제외한 나머지 설정 디폴트 값 사용
*   지점필터로 인한 분석 지점개수 재입력
*   for 문을 통해 지점번호, 위도, 경도, 입력자료_u값, 입력자료_v값, 분류결과 파일로 출력
*   출력값 동일하게 터미널로 확인






In [ ]:
#-------------------------------------------------------------------------
# .. Set Model

kmeans = KMeans(n_clusters=n_clst, random_state=0).fit(re_avg_nor_tran_y)

# .. print file
num_stn = re_avg_nor_tran_y.shape[0]

with open( out_file, 'w' ) as f:
     for i in range(num_stn):
          # .. for num_ele = 1
          #print ( re_obs_stn_id[i][0], re_obs_stn_lat[i][0], re_obs_stn_lon[i][0], kmeans.labels_[i],
          #        sep=',', file=f)
          # .. for num_ele = 2
          print ( re_obs_stn_id[i][0], re_obs_stn_lat[i][0], re_obs_stn_lon[i][0],
                  re_avg_nor_tran_y[i][0], re_avg_nor_tran_y[i][1], kmeans.labels_[i], sep=',', file=f)

for i in range(num_stn):
    print ( re_obs_stn_id[i][0], re_obs_stn_lat[i][0], re_obs_stn_lon[i][0], kmeans.labels_[i],
            sep=',')

### **결과 예시**
cvs 포맷으로 출력

47090,38.25085,128.56473,0.48837212,0.7542534,3

47093,37.947379999999995,127.75443,0.486916,0.75637555,3

47095,38.147870000000005,127.3042,0.4876144,0.75541806,3
